In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import tensorflow as tf
import numpy as np


#  Import and read the data.csv.
import pandas as pd 
data_df = pd.read_csv("archive/data.csv")
#data_df.head()

In [2]:
#Drop unnecessary columns
data_df = data_df.drop(["Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Games","City", "Nat_Pop", "GDP"], 1)

#data_df = data_df.drop(["GDP_Per_Cap"], 1)
data_df = data_df[data_df.GDP_Per_Cap != '#DIV/0!']

#Rename Global_Pop% and Rural_Pop% to make it easier to call in future
data_df.rename(columns = {'Global_Pop%':'Global_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Rural_Pop%':'Rural_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Global_GDP%':'Global_GDP_Perc'}, inplace = True)

C:\Users\14043\AppData\Local\Temp/ipykernel_17368/850627988.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Games","City", "Nat_Pop", "GDP"], 1)


In [3]:
#Filter out season data to just summer
data_df = data_df[data_df["Season"].str.contains("Winter") == False]

In [4]:
#Turn Medal Values into Boolean
data_df.fillna(value = False,
          inplace = True)

#Drop Season since no longer needed after filtering
data_df = data_df.drop(["Season"],1)
data_df.head(10)

C:\Users\14043\AppData\Local\Temp/ipykernel_17368/1206795605.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Season"],1)


,Unnamed: 0,Team,NOC,Year,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,0,China,CHN,1992,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,1,China,CHN,2012,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,26,Finland,FIN,2000,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,29,Finland,FIN,2000,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,50,Norway,NOR,2008,Handball,Gold,0,0.000705,21.474,80.592683,97101.38727,0.007291
51,51,Romania,ROU,2016,Weightlifting,False,0,0.002648,45.251,75.012195,9540.579969,0.00248
52,52,Norway,NOR,2000,Wrestling,False,0,0.000734,23.919,78.634146,38076.43209,0.005089
53,53,Norway,NOR,2004,Wrestling,False,0,0.000713,22.725,79.841463,57492.41601,0.006027
58,58,Netherlands,NLD,1984,Water Polo,False,0,0.003030,33.681,76.233171,9844.559262,0.011736
59,59,Netherlands,NLD,1988,Hockey,Bronze,0,0.002891,32.094,76.890244,17547.31372,0.01356


In [5]:
#Replace medal values to true to finish boolean
data_df = data_df.replace({"Gold" : True})
data_df = data_df.replace({"Silver" : True})
data_df = data_df.replace({"Bronze" : True})

In [6]:
data_df.groupby('NOC')


In [7]:
#data_df = data_df.drop(["Host","Global_Pop_Perc","Rural_Pop_Perc","Life_Expect","GDP_Per_Cap","Global_GDP_Perc"],1)
data_df.head(10)

,Unnamed: 0,Team,NOC,Year,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,0,China,CHN,1992,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,1,China,CHN,2012,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,26,Finland,FIN,2000,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,29,Finland,FIN,2000,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,50,Norway,NOR,2008,Handball,True,0,0.000705,21.474,80.592683,97101.38727,0.007291
51,51,Romania,ROU,2016,Weightlifting,False,0,0.002648,45.251,75.012195,9540.579969,0.00248
52,52,Norway,NOR,2000,Wrestling,False,0,0.000734,23.919,78.634146,38076.43209,0.005089
53,53,Norway,NOR,2004,Wrestling,False,0,0.000713,22.725,79.841463,57492.41601,0.006027
58,58,Netherlands,NLD,1984,Water Polo,False,0,0.003030,33.681,76.233171,9844.559262,0.011736
59,59,Netherlands,NLD,1988,Hockey,True,0,0.002891,32.094,76.890244,17547.31372,0.01356


In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
NOC_group = data_df.groupby(["NOC", "Sport","Team"])

# mean, median, variance, standard deviation, and SEM of the tumor volume. 
NOC_sum = NOC_group["Medal"].count()
NOC_sum

NOC  Sport          Team    
ABW  Athletics      Aruba        9
     Boxing         Aruba        2
     Cycling        Aruba        3
     Fencing        Aruba        2
     Judo           Aruba        5
                                ..
ZWE  Shooting       Zimbabwe    21
     Swimming       Zimbabwe    67
     Tennis         Zimbabwe    19
     Triathlon      Zimbabwe     3
     Weightlifting  Zimbabwe     1
Name: Medal, Length: 3174, dtype: int64

In [9]:
NOC_df = pd.DataFrame(NOC_sum)
NOC_df.reset_index(inplace=True)
NOC_df

,NOC,Sport,Team,Medal
0,ABW,Athletics,Aruba,9
1,ABW,Boxing,Aruba,2
2,ABW,Cycling,Aruba,3
3,ABW,Fencing,Aruba,2
4,ABW,Judo,Aruba,5
...,...,...,...,...
3169,ZWE,Shooting,Zimbabwe,21
3170,ZWE,Swimming,Zimbabwe,67
3171,ZWE,Tennis,Zimbabwe,19
3172,ZWE,Triathlon,Zimbabwe,3


In [10]:
NOC_df = NOC_df.pivot_table('Medal', ['NOC', 'Team'], 'Sport')
NOC_df.reset_index(inplace=True)
NOC_df.head()

Sport,NOC,Team,Archery,Athletics,Badminton,Baseball,Basketball,Beach Volleyball,Boxing,Canoeing,...,Synchronized Swimming,Table Tennis,Taekwondo,Tennis,Trampolining,Triathlon,Volleyball,Water Polo,Weightlifting,Wrestling
0,ABW,Aruba,NaN,9.0,NaN,NaN,NaN,NaN,2.0,NaN,...,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN
1,AFG,Afghanistan,NaN,19.0,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,42.0
2,AGO,Angola,NaN,29.0,NaN,NaN,71.0,2.0,7.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,4.0
4,AND,Andorra,NaN,13.0,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
NOC_df.to_csv(r'archive\countryMedalCount.csv', index = False)